# YAHOO電影爬蟲練習
## 這個作業主要是練習爬取YAHOO電影的資訊，主要會用到基本的GET method，以及解析伺服器回傳的html，從中擷取我們想要的資料。

### 這個範例示範如何爬取上映中的電影資訊。先訪問YAHOO電影首頁：https://movies.yahoo.com.tw ，點選「上映中」的分頁，先確認上映中的電影總共有幾筆，然後逐頁爬取。

In [1]:
import requests
import re
from bs4 import BeautifulSoup

In [6]:
# 先觀察一下目前上映中的電影數量
url = 'https://movies.yahoo.com.tw/movie_intheaters.html'
resp = requests.get(url)
resp.encoding = 'utf-8'

soup = BeautifulSoup(resp.text, 'lxml')
html = soup.find("div", attrs={'class':'release_box'})  # 尋找正在上映中的全部電影筆數，其所在的tag
print("正在上映中總共: ", html.p.string)

正在上映中總共:  共96筆，目前顯示1~10筆


### 爬取所有上映中的電影資料：電影名稱、期待度、滿意度、簡介等等。

In [27]:
# 接下來要爬取每一頁的電影介紹
# 先點選網頁最底部的下一頁，觀察其網址變化。然後我們把頁數當作變數傳入，用迴圈爬取。
max_page = 5
for page_number in range(1, max_page+1, 1):
    url = 'https://movies.yahoo.com.tw/movie_intheaters.html'
    payload = {'page':str(page_number)}
    resp = requests.get(url, params=payload)
    resp.encoding = 'utf-8'
    soup = BeautifulSoup(resp.text, 'lxml')

    # 把電影介紹擷取出來
    movie_list = soup.find("ul", attrs={"class":"release_list"}).find_all("li")
    for p in movie_list:
        try:
            print("----------------------------------------------------------------------")
            # 電影名稱
            movie_name = p.find("div", attrs={"class":"release_movie_name"})
            print("電影名稱：", movie_name.a.string)
            
            # 定位電影評價資訊欄位
            level_box = movie_name.find("dl", attrs={"class":"levelbox"})
            
            # 期待度
            expectation = level_box.find("div", attrs={"class":"leveltext"})
            print("期待度：", expectation.span.string)
            
            # 滿意度
            satisfaction = level_box.find_all("div", attrs={"class":"leveltext"})[1]
            print("滿意度：", satisfaction.span["data-num"])  # 滿意度可以從"data-num"這個屬性擷取
            
            # 簡介
            movie_info = p.find("div", attrs={"class":"release_text"})
            print(movie_info.span.string)
        except:
            print('none')


----------------------------------------------------------------------
電影名稱： 
                  大紅狗克里弗
期待度： 87%
滿意度： 3.7

                  超級巨大的毛茸茸英雄即將登上大銀幕！
 
改編自備受喜愛的經典童書，大紅狗克里弗將會教導全世界如何付出大大的愛！
 
當中學生艾蜜莉伊莉莎白（達比坎普 飾）遇見一名擁有魔法的動物救援者（約翰克里斯 飾），而他又送給她一隻紅色的小狗狗，她絕對沒想到隔天一覺醒來，竟然會在她位於紐約市的小公寓，發現一隻三米高的巨大獵犬。因為她的單親媽媽（席安娜蓋蘿莉 飾）正在出差，於是艾蜜莉和她愛玩又孩子氣的凱西叔叔（傑克懷特霍爾 飾）就在被暱稱為大蘋果的紐約市，展開一場驚險刺激的冒險旅程，保證令人看得大呼過癮。
                
----------------------------------------------------------------------
電影名稱： 
                  寶可夢：皮卡丘與可可的冒險
期待度： 81%
滿意度： 4.2

                  小智和皮卡丘將前往有寶可夢樂園之稱的奥可雅森林，並和由薩戮德撫養長大的人類孩子「可可」相遇。第一次交到了「人類朋友」讓一直深信自己是寶可夢的可可心中產生了一絲懷疑及煩惱，即使如此，可可還是決定和小智及皮卡丘攜手共同對抗打亂奧可雅森林平靜生活的不速之客。
                
----------------------------------------------------------------------
電影名稱： 
                  麥迪遜之橋
期待度： 55%
滿意度： 4.1

                  《麥迪遜之橋》由克林伊斯威特執導，並與梅莉史翠普擔任主演。故事背景設定於60年代的美國，攝影師勞勃（克林伊斯威特　飾演）接受《國家地理雜誌》的任務，到愛荷華州進行拍攝，碰巧向已婚的法蘭西斯（梅莉史翠普　飾演）問路，兩人迅速墜入愛河，一起度過人生中難忘的四天，戀情雖然短暫但都對雙方產生了深遠影響。
               

----------------------------------------------------------------------
電影名稱： 
                  獵魂者
期待度： 71%
滿意度： 3.8

                  ★ 《極惡對決》製作團隊新作 2021年終最強韓國犯罪動作鉅片
★ 《犯罪都市》尹啟相 《人間中毒》林知衍 共演年度壓軸之作
★ 我是誰？大場面動作超越《神鬼認證》懸疑直逼《記憶拼圖》
★　每12個小時 換一個身體 分秒必爭 解救自身危機
★　只有我能獵殺我自己！拼湊靈魂記憶 揭開不為人知的恐怖陰謀
 
一名男子（尹啟相 飾）車禍後不僅失去記憶，甚至每隔12小時，靈魂就會移轉到另一人身上。在他拼命尋找「我是誰」的過程中，不但遭到地下組織追殺，更遇見一位自稱認識他的神祕女子（林知衍 飾）。疑點重重的記憶謎團、分秒必爭的獵殺行動，他能否找回自己的身體，並揭發不可告人的真相？
                
----------------------------------------------------------------------
電影名稱： 
                  魔法少女☆伊莉雅：LICHT無名的少女
期待度： 73%
滿意度： 3.4

                  ★使用Fate/ Say Night人氣角色：伊莉雅與美遊為主角，原著作者廣山弘在原有的世界觀中，再建構出平行世界風格的原創作品，推出後大受好評。
★繼續沿用Fate/ Stay Night 故事架構與登場人物，故事改為魔法少女風，戰鬥場面依舊熱血！
 
在與恩茲華斯的激烈戰鬥之後，眾人獲得了短暫的喘息空間。伊莉雅一行人在救出美遊之後，從平行世界的衛宮士郎口中得知美遊的過去，以及在這個世界中發動的「聖杯戰爭」。
那不僅僅只是美遊的悲慘命運，更是哥哥士郎為了美遊而賭上性命，進行一場壯烈戰鬥的偉大故事。為了拯救人類，恩茲華斯決定犧牲美遊。為了保護美遊，士郎不惜與世界以及恩茲華斯為敵。面對相互對立的兩種正義…，伊莉雅立誓要找出方法，守護這一切！
 
為了創造能與夥伴共同歡笑的未來，魔法少女們的戰鬥將再次展開。
                
------------------------------

----------------------------------------------------------------------
電影名稱： 
                  電影之神
期待度： 96%
滿意度： 4.4

                  ★ 日本電影大廠「松竹映畫」100周年紀念作品！
★「日本愛情小說大賞」得主原田舞葉同名小說改編！
★「日本殿堂級名導」山田洋次，引領觀眾追憶日本電影黃金歲月！
★ 集結日本影帝、影后頂尖卡司，共同展開跨越時代的愛情與友誼！
★ 一部獻給熱愛電影人們的真情力作，即將席捲日本獎季主要獎項！
★ 在虛實交織的幻夢之中，讓電影之神延續不離不棄的眷顧！ 
★《ALWAYS幸福的三丁目》名導山崎貴擔綱全片特效監修！
★ 今昔虛實交錯間，致敬《風雲人物》、《開羅紫玫瑰》等影史經典！
★ 感動入選金馬影展、北京電影節、上海電影節、香港夏日電影節！
★「日本大衛鮑伊」澤田研二，息影多年接手好友志村健重返大銀幕！
★「日本奧斯卡影帝」菅田將暉領銜主演，變身逐夢熱血電影青年！
★「日劇學院賞視后」永野芽郁化身食堂之女，展現溫柔婉約風情！
★「日劇學院賞視后」北川景子化身昭和時代氣質女星！
★ 人氣樂團「RADWIMPS」主唱野田洋次郎加盟演出！
★ 野田洋次郎攜手菅田將暉共同演唱感人主題曲〈泡沫之歌〉！
 
你我未完成的故事，都存在於每顆鏡頭之間
嗜賭酗酒更積欠一屁股債的鄉（澤田研二∕菅田將暉分飾），年邁的他只剩電影可以喜愛。面對失望的家人沒收他一切財物、不准他再隨意揮霍之後，他躲去昔日片廠工作伙伴寺新（小林稔侍∕野田洋次郎分飾）經營的社區戲院，沒想到竟一頭栽進年輕時所熱愛的電影舊夢當中。原來，他過去曾任名導助理，並邂逅氣質女星園子（北川景子飾演），甚至跟寺新共同愛上在食堂工作的淑子（永野芽郁∕宮本信子分飾）。當鄉終於有機會拍自己的電影時，一場意外卻讓機會化為泡影。直到半世紀以後，當初所要拍攝的劇本，竟再度出現在年邁的鄉眼前，也改寫他一生未完成的電影之夢…。
 
【關於電影】
 
集結日本影帝、影后頂尖卡司，一部獻給熱愛電影人們真情力作！
 
日本電影大廠「松竹映畫」，是見證日本電影發展史的重要公司。打從1920年「松竹電影合名社」成立，以及「蒲田製片場」開幕，松竹不斷為日本影史帶來眾多傑作與賣座鉅片，業務包辦電影